# Rerank
本筆記本介紹如何開始使用Chroma向量儲存。
- https://python.langchain.com/docs/integrations/document_transformers/rankllm-reranker/

In [ ]:
# 安裝套件
!uv pip install -qU "langchain-chroma"

In [ ]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
# llm模型設定
# https://build.nvidia.com/deepseek-ai/deepseek-r1
# nvapi-xxx
import getpass
import os
if not os.environ.get("NVIDIA_API_KEY"):
  os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA: ")

from langchain.chat_models import init_chat_model
llm = init_chat_model("meta/llama-4-maverick-17b-128e-instruct", model_provider="nvidia")

In [ ]:
# embedding and rerank 模型
# https://jina.ai/
# jina_xxx
import getpass
import os

if not os.environ.get("JINA_API_KEY"):
  os.environ["JINA_API_KEY"] = getpass.getpass("Enter API key for Voyage AI: ")

from langchain_community.embeddings import JinaEmbeddings
embeddings = JinaEmbeddings(
    jina_api_key=os.environ["JINA_API_KEY"], model_name="jina-embeddings-v3"
)


from langchain_community.document_compressors import JinaRerank

rerank = JinaRerank(model="jina-reranker-v2-base-multilingual")  # 或使用 "rerank-1" 精準但較慢

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    #collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db30",  # Where to save data locally, remove if not necessary
)

In [ ]:
# 輸出所有儲存的 document（包括 id, page_content, metadata）
all_data = vector_store._collection.get()

# 印出每筆紀錄的內容
for i in range(len(all_data["ids"])):
    print(f"ID: {all_data['ids'][i]}")
    print(f"Document: {all_data['documents'][i]}")
    print(f"Metadata: {all_data['metadatas'][i]}")
    print("="*40)

In [ ]:
# Query by turning into retriever

retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10}
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

In [ ]:
# Step 2: 把向量資料庫轉換為 retriever，並指定檢索參數
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
query = "RStealing from the bank is a crime"
docs = retriever.invoke(query)
pretty_print_docs(docs)

In [ ]:
query = "Stealing from the bank is a crime"
from langchain.retrievers import ContextualCompressionRetriever
rerank_retriever = ContextualCompressionRetriever(
    base_compressor=rerank, base_retriever=retriever
)

rerank_docs = rerank_retriever.invoke(query)
#print(rerank_docs)
pretty_print_docs(rerank_docs)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate



system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(rerank_retriever, question_answer_chain)
query = "Stealing from the bank is a crime"
chain.invoke({"input": query}, filter={"source": "news"})


##   作業

**使用 Rerank 提升問答系統**

Notebook 中創建了一個基本的問答系統，使用 Chroma 檢索相關文檔並使用 LLM 回答問題。

**作業：**

1.  **擴展問答系統：**
    * 在 notebook 的基礎上，加入 Rerank 功能，提升系統回答問題的準確性。
2.  **選擇文檔來源：**
    * 選擇一個適合問答系統的文檔來源，例如：
        * 網頁文章
        * 產品說明書
        * 法律文件
        * 學術論文
    * 請在報告中說明你選擇的文檔來源及其適用情境。
3.  **實作 Rerank：**
    * 實作 Rerank 功能，對向量資料庫檢索到的文檔進行重新排序。
    * 可以選擇以下 Rerank 方法：
        * 自訂 Rerank 演算法
    * 請在報告中說明你選擇的 Rerank 方法及其原因。
4.  **回答使用者問題：**
    * 系統能夠根據 Rerank 後的文檔，回答使用者提出的問題。

**評估標準：**

* 系統是否能夠正確回答使用者提出的問題？ (40%)
* Rerank 是否能夠提升檢索結果的相關性？ (30%)
* Rerank 對系統效率的影響？ (30%)
